In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, TimeDistributed
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Load and merge datasets
file_paths = {
    "thumbup": "../Data_Extracting/emg_training_data_thumbup_time_series.csv",
    "fist": "../Data_Extracting/emg_training_data_fist_time_series.csv",
}

data_frames = []
for gesture, path in file_paths.items():
    df = pd.read_csv(path)
    df["gesture"] = gesture
    data_frames.append(df)

data = pd.concat(data_frames, ignore_index=True)

# Extract features and labels
X = data.iloc[:, :-1].values  # Exclude the label column
y = data.iloc[:, -1].values   # Label column

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Normalize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape for LSTM and CNN+LSTM (samples, time_steps=50, features=6)
X_reshaped = X_scaled.reshape(len(X_scaled), 50, 6)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_encoded, test_size=0.2, random_state=42)


In [4]:
# LSTM Model
lstm_model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(50, 6)),
    LSTM(64, return_sequences=False),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(label_encoder.classes_), activation='softmax')
])

lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train LSTM
history_lstm = lstm_model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test))

# Evaluate LSTM
y_lstm_pred = np.argmax(lstm_model.predict(X_test), axis=1)
print(f"LSTM Accuracy: {accuracy_score(y_test, y_lstm_pred):.4f}")
print(classification_report(y_test, y_lstm_pred, target_names=label_encoder.classes_))

C:\Users\Rohan Datta\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - accuracy: 0.5993 - loss: 0.6535 - val_accuracy: 0.5814 - val_loss: 0.4835
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.7225 - loss: 0.4238 - val_accuracy: 0.9302 - val_loss: 0.1441
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.6866 - loss: 1.0313 - val_accuracy: 0.6279 - val_loss: 0.6684
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.5351 - loss: 0.7423 - val_accuracy: 0.6279 - val_loss: 0.6712
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.5625 - loss: 0.6987 - val_accuracy: 0.6279 - val_loss: 0.6719
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.5453 - loss: 0.6958 - val_accuracy: 0.5581 - val_loss: 0.6860
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.5760 - loss: 0.6820 - val_accuracy: 0.6279 - val_loss: 0.6625
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.5213 - loss: 0.6929 - val_accuracy: 0.9302 - v

In [5]:
# CNN + LSTM Hybrid Model
hybrid_model = Sequential([
    TimeDistributed(Conv1D(64, kernel_size=3, activation='relu'), input_shape=(50, 6, 1)),
    TimeDistributed(MaxPooling1D(pool_size=2)),
    TimeDistributed(Flatten()),
    
    LSTM(64, return_sequences=False),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(label_encoder.classes_), activation='softmax')
])

hybrid_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train Hybrid Model
history_hybrid = hybrid_model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test))

# Evaluate Hybrid Model
y_hybrid_pred = np.argmax(hybrid_model.predict(X_test), axis=1)
print(f"CNN+LSTM Accuracy: {accuracy_score(y_test, y_hybrid_pred):.4f}")
print(classification_report(y_test, y_hybrid_pred, target_names=label_encoder.classes_))


Epoch 1/50


C:\Users\Rohan Datta\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 11s 127ms/step - accuracy: 0.4288 - loss: 0.7124 - val_accuracy: 0.6279 - val_loss: 0.6174
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.5149 - loss: 0.6508 - val_accuracy: 0.7907 - val_loss: 0.4406
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.8317 - loss: 0.4559 - val_accuracy: 0.6279 - val_loss: 1.5638
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.5895 - loss: 1.0549 - val_accuracy: 0.4419 - val_loss: 0.7682
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.5454 - loss: 0.7404 - val_accuracy: 0.6279 - val_loss: 0.6461
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.5790 - loss: 0.6879 - val_accuracy: 0.8140 - val_loss: 0.6291
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.6510 - loss: 0.6254 - val_accuracy: 0.8372 - val_loss: 0.5248
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.6401 - loss: 0.6362 - val_accuracy: 0.9302 - val_loss: 